# Microsoft Movie Data Analysis
## Problem Overview
The purpose of this EDA is to determine the types or genres of movies that are most in demand and are performing best in the market in order to come up with actionable insights for the company's stake holders.

## Business Understanding

Big companies have been recently flooding into the home entertainment industry of which movies are a big part. In this analysis we are going to look at data from sources such as IMDB the popular movie review site. We are looking to determine:
* Most popular genres of movies
* Which genres gross the most domestically
* Which genres gross the most worldwide
* Trend in how much movies gross over the years
* Trend in movie ratings over time

## Data Understanding

The first thing we shall do is import the necessary libraries and store our data in some pandas dataframes. From there we can use some descriptive methods to understand our data.

##### NB: The project directory contains a folder Data which contains the data files. Some files that are too large to push to github shall 
#####          be left compressed to be uncompressed when running this notebook

In [1]:
#modules for uncompressing some data files
import zipfile
import os

#path to compressed file
zip_file_imdb = 'Data/im.db.zip'

#path to extracted file
extracted_file_imdb = 'Data/im.db'

#check if extracted file already exists
if not os.path.isfile(extracted_file_imdb):
    #extract compressed file
    with zipfile.ZipFile(zip_file_imdb, 'r') as zip_reference:
        zip_reference.extractall('Data')

In [2]:
!cd Data && ls -a && cd ..

.			im.db		      rt.reviews.tsv.gz
..			im.db.zip	      tmdb.movies.csv.gz
bom.movie_gross.csv.gz	rt.movie_info.tsv.gz  tn.movie_budgets.csv.gz


In [3]:
import pandas as pd
import numpy as np
import sqlite3

#connection and cursor to database
conn = sqlite3.connect('Data/im.db')
cursor = conn.cursor()

Lets see what tables are in our ``im.db`` database.

In [4]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [5]:
movie_basics = pd.read_sql("SELECT * FROM movie_basics;", conn)
print("shape:", movie_basics.shape)
movie_basics.head()

shape: (146144, 6)


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [6]:
movie_ratings = pd.read_sql("SELECT * FROM movie_ratings;", conn)
print("shape:", movie_ratings.shape)
movie_ratings.head(3)

shape: (73856, 3)


,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20


The cells above read necessary tables from the database into dataframes. 

In [7]:
movie_gross = pd.read_csv('Data/bom.movie_gross.csv.gz')
print("shape:", movie_gross.shape)
movie_gross.head(3)

shape: (3387, 5)


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010


In [8]:
movie_budgets = pd.read_csv('Data/tn.movie_budgets.csv.gz')
print("shape:", movie_budgets.shape)
movie_budgets.head(3)

shape: (5782, 6)


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"


Ok. We have our dataframes:
``movie_basics`` ``movie_gross``
``movie_ratings`` ``movie_budgets``

## Data Preparation
### Data Cleaning

#### 1.Lets normalize column names
This will make merging easier

In [9]:
movie_gross.rename(columns={'title':'primary_title'}, inplace=True)
movie_gross.columns

Index(['primary_title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')

In [10]:
movie_budgets.rename(columns={'movie':'primary_title'}, inplace=True)
movie_budgets.columns

Index(['id', 'release_date', 'primary_title', 'production_budget',
       'domestic_gross', 'worldwide_gross'],
      dtype='object')

#### 2.Missing Values.

In [11]:
print("movie_basics nulls:\n", movie_basics.isna().sum().loc[movie_basics.isna().sum()>0])
print("movie_ratings nulls:\n", movie_ratings.isna().sum().loc[movie_ratings.isna().sum()>0])
print("movie_gross nulls:\n", movie_gross.isna().sum().loc[movie_gross.isna().sum()>0])
print("movie_budgets nulls:\n", movie_budgets.isna().sum().loc[movie_budgets.isna().sum()>0])

movie_basics nulls:
 original_title        21
runtime_minutes    31739
genres              5408
dtype: int64
movie_ratings nulls:
 Series([], dtype: int64)
movie_gross nulls:
 studio               5
domestic_gross      28
foreign_gross     1350
dtype: int64
movie_budgets nulls:
 Series([], dtype: int64)


In [12]:
#fill null values in 'runtime_minutes' column with the mean value
movie_basics['runtime_minutes'].fillna(movie_basics['runtime_minutes'].mean(), inplace=True)
#Remove all remaining rows with null values
movie_basics.dropna(axis=0, inplace=True)
movie_basics.isna().sum()

movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
dtype: int64

For the ``movie_gross`` dataframe, we are going to create two subsets of the dataframe so that one contains only domestic movie grossing and the other contains foreign grossing with null values eliminated.

In [13]:
movie_domestic_gross =  movie_gross.loc[:, movie_gross.columns != 'foreign_gross']
movie_foreign_gross = movie_gross.loc[:, movie_gross.columns != 'domestic_gross']
print("domestic_null:\n", movie_domestic_gross.isna().sum(),"\nforeign_null:\n", movie_foreign_gross.isna().sum())

domestic_null:
 primary_title      0
studio             5
domestic_gross    28
year               0
dtype: int64 
foreign_null:
 primary_title       0
studio              5
foreign_gross    1350
year                0
dtype: int64


In [14]:
movie_domestic_gross.dropna(inplace=True)
movie_foreign_gross.dropna(inplace=True)
print("domestic:", movie_domestic_gross.shape)
print("foreign:", movie_foreign_gross.shape)

domestic: (3356, 4)
foreign: (2033, 4)


<ipython-input-14-50876856e023>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_domestic_gross.dropna(inplace=True)
<ipython-input-14-50876856e023>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_foreign_gross.dropna(inplace=True)


#### 3.Duplicates

In [15]:
print("movie_basics dups:\n", movie_basics.duplicated().sum())
print("movie_ratings dups:\n", movie_ratings.duplicated().sum())
print("movie_domestic_gross dups:\n", movie_domestic_gross.duplicated().sum())
print("movie_foreign_gross dups:\n", movie_foreign_gross.duplicated().sum())
print("movie_budgets dups:\n", movie_budgets.duplicated().sum())

movie_basics dups:
 0
movie_ratings dups:
 0
movie_domestic_gross dups:
 0
movie_foreign_gross dups:
 0
movie_budgets dups:
 0


We have no duplicate values. Lets store our dataframe names in a list called ``movie_df_names``

In [16]:
movie_df_names=[movie_basics, movie_ratings, movie_domestic_gross, movie_foreign_gross, movie_budgets]

In [17]:
# a function to print out head()'s in our dfs
def print_heads(df_names):
    for df in df_names:
        print(str(df.columns), "\n", df.head(2), "\n")
print_heads(movie_df_names)

Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object') 
     movie_id                    primary_title   original_title  start_year  \
0  tt0063540                        Sunghursh        Sunghursh        2013   
1  tt0066787  One Day Before the Rainy Season  Ashad Ka Ek Din        2019   

   runtime_minutes              genres  
0            175.0  Action,Crime,Drama  
1            114.0     Biography,Drama   

Index(['movie_id', 'averagerating', 'numvotes'], dtype='object') 
      movie_id  averagerating  numvotes
0  tt10356526            8.3        31
1  tt10384606            8.9       559 

Index(['primary_title', 'studio', 'domestic_gross', 'year'], dtype='object') 
                 primary_title studio  domestic_gross  year
0                 Toy Story 3     BV     415000000.0  2010
1  Alice in Wonderland (2010)     BV     334200000.0  2010 

Index(['primary_title', 'studio', 'foreign_gross', 'year'], dtype='

## Analysis

## Summary